# **Incrémentation du RAG avec fichier excel pour un chatbot spécialisé en cuisine et nutrition**

### Recupération des fichiers sur le drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Faire les imports nécessaires au projet

In [ ]:
# Install packages (restart runtime after)
! pip install -q accelerate bitsandbytes langchain langchain-community sentence-transformers  openpyxl pacmap pypdf
! pip install faiss-cpu --no-cache

# Import this package for embedders
! pip install -U langchain-huggingface

# Use langchain for text splitter (transform)
! pip install -qU langchain-text-splitters

# Install packages
! pip install --quiet langchain_experimental
! pip install langchain-community
! pip install sentence_transformers

# Reranking

!pip install xlrd==1.2.0

## **1. Recuperer les données excel**

In [ ]:
import xlrd
from typing import List
from langchain.docstore.document import Document

class ExcelFileLoader:
    """Loads an Excel file and maps rows to column headers."""

    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self) -> List[Document]:
        workbook = xlrd.open_workbook(self.file_path)
        sheet = workbook.sheet_by_index(0)

        # Headers from first row
        headers = sheet.row_values(0)
        documents = []

        for row_index in range(1, sheet.nrows):  # Skip headers
            row = sheet.row_values(row_index)
            content_lines = [f"{header.strip()} : {value}" for header, value in zip(headers, row)]
            content = "\n".join(content_lines)
            metadata = {"row_number": row_index + 1}
            documents.append(Document(page_content=content, metadata=metadata))

        return documents

# Path to your Excel file
file_path = "/content/drive/MyDrive/SAE_IA/SAE S2/Table Ciqual_version_courte.xls"

# Load documents
loader = ExcelFileLoader(file_path=file_path)
docs_before_split = loader.load()

# Calculate average document length
avg_doc_length = lambda docs: sum(len(doc.page_content) for doc in docs) // len(docs)

# Display stats
print(f"Nombre de documents (lignes de données) : {len(docs_before_split)}")
print(f"Longueur moyenne par document (en caractères) : {avg_doc_length(docs_before_split)}")


Nombre de documents (lignes de données) : 2848
Longueur moyenne par document (en caractères) : 1323


In [ ]:
print(docs_before_split[11].page_content)

alim_grp_code : 02
alim_ssgrp_code : 0201
alim_ssssgrp_code : 020101
alim_grp_nom_fr : fruits, légumes, légumineuses et oléagineux
alim_ssgrp_nom_fr : légumes
alim_ssssgrp_nom_fr : légumes crus
alim_code : 20026.0
alim_nom_fr : Endive, crue
alim_nom_sci : 
Energie, Règlement UE N° 1169/2011 (kJ/100 g) : 84,8
Energie, Règlement UE N° 1169/2011 (kcal/100 g) : 20,2
Energie, N x facteur Jones, avec fibres  (kJ/100 g) : 84,8
Energie, N x facteur Jones, avec fibres  (kcal/100 g) : 20,2
Eau (g/100 g) : 94,3
Protéines, N x facteur de Jones (g/100 g) : 1,19
Protéines, N x 6.25 (g/100 g) : 1,19
Glucides (g/100 g) : 2,83
Lipides (g/100 g) : < 0,5
Sucres (g/100 g) : 2,4
Fructose (g/100 g) : 0,9
Galactose (g/100 g) : -
Glucose (g/100 g) : 1,5
Lactose (g/100 g) : < 0,2
Amidon (g/100 g) : < 0,35
Fibres alimentaires (g/100 g) : 1,1
Alcool (g/100 g) : 0
Acides organiques (g/100 g) : 0,02
AG saturés (g/100 g) : < 0,01
AG monoinsaturés (g/100 g) : < 0,01
Cholestérol (mg/100 g) : 0
Sel chlorure de sodium 

In [ ]:
max_char = max(len(doc.page_content) for doc in docs_before_split)
print(f"Taille max d'un doc : {max_char} caractères")



Taille max d'un doc : 1486 caractères


## **2. Segmenter le texte**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=64,
    separators=["\n\n", "\n", ";", ".", " "]  # priorité à la coupure sur les lignes ou les points-virgules
)

# Return list
docs_after_split = text_splitter.split_documents(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)
print(f'After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split} (average chunk length).')

After split, there were 8871 documents (chunks), with average characters equal to 458 (average chunk length).


In [ ]:
! pip install hf_xet

# **3. Implementer la base de données FAISS**

In [ ]:
# 1. Configuration de l'embedder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import torch
from langchain.vectorstores import FAISS
import faiss

EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-small"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)

# 2. Charger la base existante
existing_db = FAISS.load_local("/content/drive/MyDrive/SAE_IA/SAE S2/cuisine_index",
                               embedding_model,
                               allow_dangerous_deserialization=True)

# 3. Ajouter les nouveaux documents à la base existante
existing_db.add_documents(docs_after_split)

# 4. Sauvegarder la base mise à jour
existing_db.save_local("/content/drive/MyDrive/test_rag/rag_excel_multillangal_small_racourcis_combine_recette")

# Test de l'embedder si nécessaire
user_query = "fait nutritif sur la pomme"
query_vector = embedding_model.embed_query(user_query)
print(f"A list of size : {len(query_vector)}")

A list of size : 384


In [ ]:
# Test the retriever, comparaison de vecteur
print(f"\nStarting retrieval for donne moi des fait nutritifs sur la pomme en generale ...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query="donne moi des fait nutritifs sur la pomme ", k=10)

print("\n==================================Top document==================================")

print(retrieved_docs[0].page_content)

print("==================================Metadata==================================")

print(retrieved_docs[1].metadata)



In [ ]:
# Test the retriever, comparaison de vecteur
print(f"\nStarting retrieval for propose moi une recette de quiche lorraine ...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query="fait nutritif sur la pomme", k=10)

print("\n==================================Retrieved Documents==================================")

for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i + 1}:")
    print(doc.page_content)
    print("==================================Metadata==================================")
    print(doc.metadata)

## 4. **Tester avec le LLM**

In [ ]:
# Add Hugging Face Token
from getpass import getpass
import os
HF_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

In [ ]:
### Load the LLM mistral

from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

READER_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1" # HuggingFaceH4/zephyr-7b-beta or meta-llama/Meta-Llama-3-8B

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
#bnb_config = None # no quantization
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config, token=HF_TOKEN, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME, token=HF_TOKEN)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Create a prompt template
from langchain.prompts import PromptTemplate
prompt_template = """Tu es un expert culinaire spécialisé dans les recettes, la nutrition, ainsi que les techniques de préparation et de cuisson. Utilise les éléments de contexte suivants pour répondre à la question à la fin. Veuillez suivre les règles suivantes :

Si vous ne connaissez pas la réponse, ne tentez pas d'en inventer une. Vous ne repondez uniquement a partir du context si la reponse ne se trouve pas dans le contexte on ne repond pas. Dites simplement : "Désolé je ne trouve pas la réponse finale."

Si vous trouvez la réponse, rédigez-la de manière claire

Context: {context}

Question: {question}

Helpful Answer:
"""

RAG_PROMPT_TEMPLATE = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)
print(RAG_PROMPT_TEMPLATE)

In [ ]:
# Invoke the LLM
user_query = "combien de sucre contient la banane ? "
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)

In [ ]:
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])
final_prompt = RAG_PROMPT_TEMPLATE.format(question=user_query, context=context)


model_inputs = tokenizer([final_prompt], return_tensors="pt").to(device) # transforme le context et question en pytorch et le mets sur le device (cpu ou gpu)
## **model_input car il faut deserialiser le modele qui est sous forme de dictionnaire
generated_ids = model.generate(**model_inputs,
                   pad_token_id=tokenizer.pad_token_id,
                   do_sample=True,
                   temperature=0.1,
                   max_new_tokens=500)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])